In [133]:
"""
Solved:
    -It's possible for train-test split to split data in such a way, that
   after encoding, X_train and X_test have different numbers of features.
   Split has to be rerun to fix it. First encode, then split again?
   BUT IT STILL HAS TO BE ENCODED AND SPLIT BEFORE STARTING CROSS VALIDATION
   AND SEQUENTIAL FEATURE SELECTION. MAYBE APPEND DURING SPLIT AND THEN SPLIT
   AGAIN?
    -Improve feature encoding to have proper ordering instead of random numbers
    which currently influence classification accuracy:
    https://datascience.stackexchange.com/questions/72343/encoding-with-ordinalencoder-how-to-give-levels-as-user-input

Fishy:

Pickup: 
    -add missing feature valeus
    -classification costs
    -sequential feature selection

''

<IPython.core.display.Javascript object>

In [37]:
## SKLEARN
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import CategoricalNB, MultinomialNB, GaussianNB
from sklearn import metrics
from sklearn.compose import make_column_transformer

import numpy as np

np.set_printoptions(suppress=True)
import pandas as pd
from platform import python_version

import scipy

import random

# works, sort of only. Possible additional commas that shouldn't be there.
%load_ext nb_black

print("Libs imported. Python version is: ", python_version())

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black
Libs imported. Python version is:  3.9.7


<IPython.core.display.Javascript object>

In [2]:
# utility functions

cols_mushroom = [
    "labels",
    "cap-shape",
    "cap-surface",
    "cap-color",
    "bruises",
    "odor",
    "gill-attachment",
    "gill-spacing",
    "gill-size",
    "gill-color",
    "stalk-shape",
    "stalk-root",
    "stalk-surface-above-ring",
    "stalk-surface-below-ring",
    "stalk-color-above-ring",
    "stalk-color-below-ring",
    "veil-type",
    "veil-color",
    "ring-number",
    "ring-type",
    "spore-print-color",
    "population",
    "habitat",
]

cols_car = ["buying", "maintenance", "doors", "passengers", "boot", "safety", "labels"]

cols_audiology = [
    "age_gt_60",
    "air",
    "airBoneGap",
    "ar_c",
    "ar_u",
    "bone",
    "boneAbnormal",
    "bser",
    "history_buzzing",
    "history_dizziness",
    "history_fluctuating",
    "history_fullness",
    "history_heredity",
    "history_nausea",
    "history_noise",
    "history_recruitment",
    "history_ringing",
    "history_roaring",
    "history_vomiting",
    "late_wave_poor",
    "m_at_2k",
    "m_cond_lt_1k",
    "m_gt_1k",
    "m_m_gt_2k",
    "m_m_sn",
    "m_m_sn_gt_1k",
    "m_m_sn_gt_2k",
    "m_m_sn_gt_500",
    "m_p_sn_gt_2k",
    "m_s_gt_500",
    "m_s_sn",
    "m_s_sn_gt_1k",
    "m_s_sn_gt_2k",
    "m_s_sn_gt_3k",
    "m_s_sn_gt_4k",
    "m_sn_2_3k",
    "m_sn_gt_1k",
    "m_sn_gt_2k",
    "m_sn_gt_3k",
    "m_sn_gt_4k",
    "m_sn_gt_500",
    "m_sn_gt_6k",
    "m_sn_lt_1k",
    "m_sn_lt_2k",
    "m_sn_lt_3k",
    "middle_wave_poor",
    "mod_gt_4k",
    "mod_mixed",
    "vmod_s_mixed",
    "mod_s_sn_gt_500",
    "mod_sn",
    "mod_sn_gt_1k",
    "mod_sn_gt_2k",
    "mod_sn_gt_3k",
    "mod_sn_gt_4k",
    "mod_sn_gt_500",
    "notch_4k",
    "notch_at_4k",
    "o_ar_c",
    "o_ar_u",
    "s_sn_gt_1k",
    "s_sn_gt_2k",
    "s_sn_gt_4k",
    "speech",
    "static_normal",
    "tymp",
    "viith_nerve_signs",
    "wave_V_delayed",
    "waveform_ItoV_prolonged",
    "p-index",
    "labels",
]

cols_wine = [
    "labels",
    "alcohol",
    "mallic-acid",
    "alcalinity",
    "ash",
    "magnesium",
    "total-phenols",
    "flavonids",
    "nonflavonid-phenols",
    "proanthocyanins",
    "color-intensity",
    "hue",
    "od-of-diluted-wines",
    "proline",
]

"""
https://archive.ics.uci.edu/ml/datasets/car+evaluation
0-5 -> data
6 -> labels
"""


def load_car():
    df_car = pd.read_csv(
        "https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data",
        header=None,
        names=cols_car,
    )
    # mappings using indexes:
    # X = df_car.loc[:, :5].values
    # y = df_car.loc[:, 6].values
    labels_col = df_car.pop("labels")
    df_car.insert(0, "labels", labels_col)
    # replace 5more in doors to 5
    # df_car.loc[df_car['doors'] == '5more', 'doors'] = '5'
    # df_car["doors"] = pd.to_numeric(df_car["doors"])
    # replace more in passengers to 5
    return df_car


"""
https://archive.ics.uci.edu/ml/datasets/mushroom
1-22 -> data
0 -> labels
"""


def load_mushroom():
    df_mushroom = pd.read_csv(
        "https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.data",
        header=None,
        names=cols_mushroom,
    )
    # index mappings
    # X = df_mushroom.loc[:, 1:].values
    # y = df_mushroom.loc[:, 0].values
    return df_mushroom


"""
https://archive.ics.uci.edu/ml/datasets/Audiology+%28Standardized%29
0:length-2 -> data
length-1 unique id (p1-p200)
length -> labels
"""


def load_audiology():
    df_audiology = pd.read_csv(
        "https://archive.ics.uci.edu/ml/machine-learning-databases/audiology/audiology.standardized.data",
        header=None,
        names=cols_audiology,
    )
    # index mapping
    # length = len(df_audiology.columns)
    # X = df_audiology.loc[:, : length - 3].values
    # y = df_audiology.loc[:, length - 1].values
    df_audiology = df_audiology.drop("p-index", axis=1)
    labels_col = df_audiology.pop("labels")
    df_audiology.insert(0, "labels", labels_col)
    return df_audiology


"""
https://www.alldatascience.com/classification/wine-dataset-analysis-with-python/
1:length -> data
0 -> labels
"""


def load_wine():
    df_wine = pd.read_csv(
        "https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data",
        header=None,
        names=cols_wine,
    )
    # index mappings
    # length = len(df_wine.columns)
    # X = df_wine.loc[:, 1:].values
    # y = df_wine.loc[:, 0].values
    return df_wine

In [78]:
# Choose dataset
# dataset = load_car()
# dataset = load_mushroom()
dataset = load_audiology()
### dataset = load_wine() # all cols numerical, doesn't work

print(dataset.info())
# print("First five records:")
# print(dataset.head())

# Extract to X and y
X_cat = dataset.loc[:, dataset.columns != "labels"]
y_cat = dataset.loc[:, "labels"]

print("Size of X: ", np.shape(X_cat))
print("Size of y: ", np.shape(y_cat))

# Generate a matrix of costs
max_cost_allowed = 10000

dataset_costs = pd.DataFrame(
    np.random.randint(0, max_cost_allowed, size=(1, np.shape(X_cat)[1])),
    columns=X_cat.columns,
)

print("Size of dataset costs: ", np.shape(dataset_costs))
print("Cost of classification on full dataset: ", dataset_costs.sum(axis=1)[0])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 70 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   labels                   200 non-null    object
 1   age_gt_60                200 non-null    object
 2   air                      200 non-null    object
 3   airBoneGap               200 non-null    object
 4   ar_c                     200 non-null    object
 5   ar_u                     200 non-null    object
 6   bone                     200 non-null    object
 7   boneAbnormal             200 non-null    object
 8   bser                     200 non-null    object
 9   history_buzzing          200 non-null    object
 10  history_dizziness        200 non-null    object
 11  history_fluctuating      200 non-null    object
 12  history_fullness         200 non-null    object
 13  history_heredity         200 non-null    object
 14  history_nausea           200 non-null    o

<IPython.core.display.Javascript object>

In [71]:
# Collectors of values

cols_one_hot = [
    "cap-shape",
    "cap-surface",
    "cap-color",
    "bruises",
    "odor",
    "gill-attachment",
    "gill-spacing",
    "gill-size",
    "gill-color",
    "stalk-shape",
    "stalk-root",
    "stalk-surface-above-ring",
    "stalk-surface-below-ring",
    "stalk-color-above-ring",
    "stalk-color-below-ring",
    "veil-type",
    "veil-color",
    "ring-number",
    "ring-type",
    "spore-print-color",
    "habitat",
    "age_gt_60",
    "airBoneGap",
    "boneAbnormal",
    "bser",
    "history_buzzing",
    "history_dizziness",
    "history_fluctuating",
    "history_fullness",
    "history_heredity",
    "history_nausea",
    "history_noise",
    "history_recruitment",
    "history_ringing",
    "history_roaring",
    "history_vomiting",
    "late_wave_poor",
    "m_at_2k",
    "m_cond_lt_1k",
    "m_gt_1k",
    "m_m_gt_2k",
    "m_m_sn",
    "m_m_sn_gt_1k",
    "m_m_sn_gt_2k",
    "m_m_sn_gt_500",
    "m_p_sn_gt_2k",
    "m_s_gt_500",
    "m_s_sn",
    "m_s_sn_gt_1k",
    "m_s_sn_gt_2k",
    "m_s_sn_gt_3k",
    "m_s_sn_gt_4k",
    "m_sn_2_3k",
    "m_sn_gt_1k",
    "m_sn_gt_2k",
    "m_sn_gt_3k",
    "m_sn_gt_4k",
    "m_sn_gt_500",
    "m_sn_gt_6k",
    "m_sn_lt_1k",
    "m_sn_lt_2k",
    "m_sn_lt_3k",
    "middle_wave_poor",
    "mod_gt_4k",
    "mod_mixed",
    "vmod_s_mixed",
    "mod_s_sn_gt_500",
    "mod_sn",
    "mod_sn_gt_1k",
    "mod_sn_gt_2k",
    "mod_sn_gt_3k",
    "mod_sn_gt_4k",
    "mod_sn_gt_500",
    "notch_4k",
    "notch_at_4k",
    "s_sn_gt_1k",
    "s_sn_gt_2k",
    "s_sn_gt_4k",
    "static_normal",
    "tymp",
    "viith_nerve_signs",
    "wave_V_delayed",
    "waveform_ItoV_prolonged",
]

cols_ordinal = [
    "buying",
    "maintenance",
    "doors",
    "passengers",
    "boot",
    "safety",
    "population",
    "air",
    "ar_c",
    "ar_u",
    "bone",
    "o_ar_c",
    "o_ar_u",
    "speech",
]


def intersection(lst1, lst2):
    return [value for value in lst1 if value in lst2]


def collect_current_one_hot_columns(argNames):
    # make list of all values and create steps for them
    return intersection(cols_one_hot, argNames)


def collect_current_ordinal_columns(argNames):
    # make list of all values and create steps for them
    return intersection(cols_ordinal, argNames)

<IPython.core.display.Javascript object>

In [72]:
# Make order of categories per each column in ordinal_columns
order_of_ordinal_categories = pd.DataFrame.from_dict(
    {
        "buying": ["low", "med", "high", "vhigh", "filler1", "filler2"],
        "maintenance": ["low", "med", "high", "vhigh", "filler1", "filler2"],
        "doors": ["2", "3", "4", "5more", "filler1", "filler2"],
        "passengers": ["2", "4", "more", "filler1", "filler2", "filler3"],
        "boot": ["small", "med", "big", "filler1", "filler2", "filler3"],
        "safety": ["low", "med", "high", "filler1", "filler2", "filler3"],
        "population": ["y", "v", "s", "n", "c", "a"],
        "air": ["normal", "mild", "moderate", "severe", "profound", "filler1"],
        "ar_c": ["absent", "normal", "elevated", "filler1", "filler2", "filler3"],
        "ar_u": ["absent", "normal", "elevated", "filler1", "filler2", "filler3"],
        "bone": ["unmeasured", "normal", "mild", "moderate", "filler1", "filler2"],
        "o_ar_c": ["absent", "normal", "elevated", "filler1", "filler2", "filler3"],
        "o_ar_u": ["absent", "normal", "elevated", "filler1", "filler2", "filler3"],
        "speech": ["unmeasured", "very_poor", "poor", "normal", "good", "very_good"],
    }
)

print("Order created.")
print(order_of_ordinal_categories)

Order created.
    buying maintenance    doors passengers     boot   safety population  \
0      low         low        2          2    small      low          y   
1      med         med        3          4      med      med          v   
2     high        high        4       more      big     high          s   
3    vhigh       vhigh    5more    filler1  filler1  filler1          n   
4  filler1     filler1  filler1    filler2  filler2  filler2          c   
5  filler2     filler2  filler2    filler3  filler3  filler3          a   

        air      ar_c      ar_u        bone    o_ar_c    o_ar_u      speech  
0    normal    absent    absent  unmeasured    absent    absent  unmeasured  
1      mild    normal    normal      normal    normal    normal   very_poor  
2  moderate  elevated  elevated        mild  elevated  elevated        poor  
3    severe   filler1   filler1    moderate   filler1   filler1      normal  
4  profound   filler2   filler2     filler1   filler2   filler2      

<IPython.core.display.Javascript object>

In [80]:
# Get current ordinal and one hot columns
total_column_list = X_cat.select_dtypes(include="object").columns
current_columns_one_hot = collect_current_one_hot_columns(total_column_list)
current_columns_ordinal = collect_current_ordinal_columns(total_column_list)

# Get common cols to feed them in proper order to ordinal encoder
index_of_common_cols = order_of_ordinal_categories.columns.intersection(
    current_columns_ordinal
)
# Convert to list
order_of_ordinal_categories_list = (
    order_of_ordinal_categories[index_of_common_cols].values.transpose().tolist()
)

# Encode the columns using one hot and ordinal encoders
ohe = OneHotEncoder()
oe = OrdinalEncoder(categories=order_of_ordinal_categories_list)

for col in X_cat:
    print(col, ": ", X_cat[col].unique())

# Create column transformer
column_transform = make_column_transformer(
    (ohe, current_columns_one_hot), (oe, current_columns_ordinal), n_jobs=1
)

# Apply transformation
encoded_X_cat = column_transform.fit_transform(X_cat)
print("Successfully encoded dataset.")

# Transform y using label encoder
le = LabelEncoder()
encoded_y_cat = le.fit_transform(y_cat)
print("Successfully encoded dataset labels.")

age_gt_60 :  ['f' 't']
air :  ['mild' 'moderate' 'severe' 'normal' 'profound']
airBoneGap :  ['f' 't']
ar_c :  ['normal' '?' 'elevated' 'absent']
ar_u :  ['normal' 'absent' '?' 'elevated']
bone :  ['?' 'mild' 'moderate' 'normal' 'unmeasured']
boneAbnormal :  ['t' 'f']
bser :  ['?' 'normal' 'degraded']
history_buzzing :  ['f' 't']
history_dizziness :  ['f' 't']
history_fluctuating :  ['f' 't']
history_fullness :  ['f' 't']
history_heredity :  ['f' 't']
history_nausea :  ['f' 't']
history_noise :  ['f' 't']
history_recruitment :  ['f' 't']
history_ringing :  ['f' 't']
history_roaring :  ['f' 't']
history_vomiting :  ['f' 't']
late_wave_poor :  ['f' 't']
m_at_2k :  ['f' 't']
m_cond_lt_1k :  ['f']
m_gt_1k :  ['f' 't']
m_m_gt_2k :  ['f' 't']
m_m_sn :  ['f' 't']
m_m_sn_gt_1k :  ['f' 't']
m_m_sn_gt_2k :  ['f' 't']
m_m_sn_gt_500 :  ['f' 't']
m_p_sn_gt_2k :  ['f']
m_s_gt_500 :  ['f' 't']
m_s_sn :  ['f' 't']
m_s_sn_gt_1k :  ['f']
m_s_sn_gt_2k :  ['f' 't']
m_s_sn_gt_3k :  ['f' 't']
m_s_sn_gt_4k :

ValueError: Found unknown categories ['?'] in column 1 during fit

<IPython.core.display.Javascript object>

In [75]:
max_seed_val = 2 ** 32 - 1

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(
    encoded_X_cat,
    encoded_y_cat,
    test_size=0.2,
    random_state=random.randrange(0, max_seed_val),
)
print("Data has been split.")

if scipy.sparse.issparse(X_train):
    X_train = X_train.toarray()
if scipy.sparse.issparse(X_test):
    X_test = X_test.toarray()

print("Encoded X: ", np.shape(encoded_X_cat))

print(
    "Train dataset contains ", X_train.shape, ", y dataset contains ", y_train.shape,
)
print(
    "Test dataset contains ", X_test.shape, ", y dataset contains ", y_test.shape,
)

Data has been split.
Encoded X:  (8124, 112)
Train dataset contains  (6499, 112) , y dataset contains  (6499,)
Test dataset contains  (1625, 112) , y dataset contains  (1625,)


<IPython.core.display.Javascript object>

In [76]:
# Create a Bayes Classifier
nbayes = CategoricalNB(min_categories=encoded_X_cat.shape[1])

# Train the model using the training sets
nbayes.fit(X_train, y_train)

# Predict the response for test dataset
y_pred = nbayes.predict(X_test)

<IPython.core.display.Javascript object>

In [77]:
# Model Accuracy, how often is the classifier correct?
print("Accuracy:", metrics.accuracy_score(y_test, y_pred) * 100, "%")
print("F1 score:", metrics.f1_score(y_test, y_pred, average="weighted") * 100, "%")

Accuracy: 94.39999999999999 %
F1 score: 94.38080895582061 %


<IPython.core.display.Javascript object>

In [ ]:
# Get cost of classification


In [ ]:
"""
useful links:

https://dev.to/codinghappinessweb/analysing-dataset-using-naive-bayes-classifier-3d7o
https://towardsdatascience.com/15-tips-and-tricks-for-jupyter-notebook-that-will-ease-your-coding-experience-e469207ac95c
https://towardsdatascience.com/guide-to-encoding-categorical-features-using-scikit-learn-for-machine-learning-5048997a5c79
https://towardsdatascience.com/naive-bayes-classifier-how-to-successfully-use-it-in-python-ecf76a995069
https://scikit-learn.org/stable/modules/naive_bayes.html#multinomial-naive-bayes
"""